In [1]:
import networkx as nx
import numpy as np
from scipy.sparse.csgraph import floyd_warshall
import scipy.io


In [17]:
# asd


# nc = np.load('./output/ad_r_test_0409/fold4_cbt.npy')
# adhd = np.load('./output/ADHD_THREE_LOSS_TRAIN/fold4_cbt.npy')
# lmci = np.load('./output/LMCI_THREE_LOSS_TRAIN/fold4_cbt.npy')
ad = np.load('./output_store/ad_L_0402/fold4_cbt.npy')  

In [20]:
def information_centrality(cbt):
    dist_matrix = floyd_warshall(cbt, directed=False)
    dist_matrix[dist_matrix == np.inf] = 0

    # Calculate the information centrality for each node
    n_nodes = cbt.shape[0]
    info_centrality = np.zeros(n_nodes)

    for i in range(n_nodes):
        # Exclude self-loops
        valid_distances = dist_matrix[i][dist_matrix[i] > 0]
        
        # Harmonic mean of distances for node i
        if valid_distances.size > 0:
            info_centrality[i] = 1 / np.mean(1 / valid_distances)
            
    return info_centrality

In [21]:
information_centrality(ad)

array([0.06173058, 0.09644106, 0.05546535, 0.05361756, 0.05697403,
       0.08925149, 0.02875332, 0.03214803, 0.03055761, 0.03101071,
       0.04145549, 0.02683333, 0.04337316, 0.04285124, 0.03104786,
       0.03808175, 0.02904036, 0.02848765, 0.03428786, 0.02954236,
       0.08768222, 0.04356448, 0.03794577, 0.02776557, 0.03196492,
       0.04898178, 0.03467098, 0.0277535 , 0.03549779, 0.03083499,
       0.02975248, 0.05320429, 0.0532043 , 0.04739563, 0.06562564])

In [22]:
def laplacian_centrality(cbt):
    total_degree_sum = np.sum(cbt)
    n=35

    # Corrected Laplacian centrality calculation, focusing on non-negative contributions
    C_Laplacian_corrected = np.zeros(n)

    for i in range(n):
        # Directly using degree might already ensure non-negativity but let's ensure the calculation is clearly non-negative
        C_Laplacian_corrected[i] = np.sum(cbt[i]) / total_degree_sum
        
        
    return C_Laplacian_corrected

In [23]:
laplacian_centrality(ad)

array([0.03648186, 0.05579307, 0.03356978, 0.03468634, 0.03521464,
       0.05154989, 0.02170198, 0.02241223, 0.02282118, 0.0223278 ,
       0.02661715, 0.02166558, 0.02682552, 0.027132  , 0.02331076,
       0.02534875, 0.02190739, 0.02170893, 0.02338143, 0.02181914,
       0.05056508, 0.02682487, 0.02519499, 0.02174788, 0.02197645,
       0.03051837, 0.02386608, 0.02227584, 0.02320823, 0.02343618,
       0.02175309, 0.03242483, 0.03242484, 0.02911962, 0.03838831])

In [33]:
def node_strengths(adjacency_matrix):
    node_strengths = np.sum(adjacency_matrix, axis=1)
    return node_strengths

In [34]:
node_strengths(ad)

array([2.1935492, 3.3546765, 2.0184538, 2.0855894, 2.1173546, 3.099546 ,
       1.3048776, 1.3475828, 1.3721715, 1.3425062, 1.6004126, 1.3026886,
       1.6129413, 1.6313691, 1.4016085, 1.5241472, 1.3172278, 1.3052955,
       1.4058577, 1.3119218, 3.0403323, 1.6129019, 1.5149019, 1.307637 ,
       1.3213804, 1.8349812, 1.4349983, 1.3393818, 1.3954437, 1.4091499,
       1.3079504, 1.9496117, 1.9496119, 1.7508788, 2.3081782],
      dtype=float32)

In [24]:
def calculate_eigenvector_centrality(matrix, max_iter=100, tol=1.0e-6):
    n = matrix.shape[0]
    # Initialize the eigenvector with equal values (e.g., 1/n)
    x = np.ones(n) / n
    
    # Normalize the matrix by its largest absolute eigenvalue to ensure convergence
    eigenvalues, _ = np.linalg.eig(matrix)
    lambda_max = np.max(np.abs(eigenvalues))
    A_norm = matrix / lambda_max
    
    # Power iteration: repeatedly update the eigenvector estimate
    for _ in range(max_iter):
        x_next = A_norm.dot(x)
        x_next /= np.linalg.norm(x_next)  # Normalize the vector
        
        # Check for convergence
        if np.linalg.norm(x_next - x) < tol:
            break
        x = x_next
    
    return x

In [27]:
calculate_eigenvector_centrality(ad)

array([0.20130963, 0.29076956, 0.18817331, 0.19168799, 0.19245245,
       0.27524268, 0.13220327, 0.13510954, 0.13798453, 0.13599833,
       0.15264579, 0.13325328, 0.15110386, 0.15770729, 0.14034796,
       0.14959036, 0.13380285, 0.13329324, 0.14044639, 0.13322882,
       0.26398768, 0.15044152, 0.14801472, 0.13316843, 0.1324338 ,
       0.17382534, 0.14329804, 0.13680594, 0.13659872, 0.14111502,
       0.13316597, 0.18006538, 0.18006539, 0.16767757, 0.21092993])

In [3]:
def normalize_matrix(connectivity_matrix):
    np.fill_diagonal(connectivity_matrix, 0)
    connectivity_matrix[np.logical_or(np.isinf(connectivity_matrix), np.isnan(connectivity_matrix))] = 0
    connectivity_matrix = connectivity_matrix.astype(float)
    
    return connectivity_matrix

In [16]:
connectivity_matrices_asd = []

for i in range(1, 155):
    mat = scipy.io.loadmat(f'data_nc_asd_L/data_nc_asd_L/ASD_LH/LHASDSub{i}.mat')
    connectivity_matrix = mat['views'][:, :, :]
    connectivity_matrix = [normalize_matrix(mat) for mat in connectivity_matrix]
    connectivity_matrices_asd.append(connectivity_matrix)

connectivity_matrices_asd = np.stack(connectivity_matrices_asd)

connectivity_matrices_control = []

for i in range(1, 186):
    mat = scipy.io.loadmat(f'data_nc_asd_L/data_nc_asd_L/NC_LH/LHNCSub{i}.mat')
    connectivity_matrix = mat['views'][:, :, :]
    connectivity_matrix = [normalize_matrix(mat) for mat in connectivity_matrix]
    connectivity_matrices_control.append(connectivity_matrix)

connectivity_matrices_control = np.stack(connectivity_matrices_control)


In [19]:
connectivity_matrices_asd = []

for i in range(1, 155):
    mat = scipy.io.loadmat(f'data_nc_asd_R/data_nc_asd_R/ASD_RH/RHASDSub{i}.mat')
    connectivity_matrix = mat['views'][:, :, :]
    connectivity_matrix = [normalize_matrix(mat) for mat in connectivity_matrix]
    connectivity_matrices_asd.append(connectivity_matrix)

connectivity_matrices_asd = np.stack(connectivity_matrices_asd)

connectivity_matrices_control = []

for i in range(1, 186):
    mat = scipy.io.loadmat(f'data_nc_asd_R/data_nc_asd_R/NC_RH/RHNCSub{i}.mat')
    connectivity_matrix = mat['views'][:, :, :]
    connectivity_matrix = [normalize_matrix(mat) for mat in connectivity_matrix]
    connectivity_matrices_control.append(connectivity_matrix)

connectivity_matrices_control = np.stack(connectivity_matrices_control)


In [22]:
connectivity_matrices_ad = []

for i in range(1, 42):
    mat = scipy.io.loadmat(f'data_ad_lmci_R/data_ad_lmci_R/AD_RH/subject{i}.mat')
    connectivity_matrix = mat['A'][:, :, :]
    connectivity_matrix = [normalize_matrix(mat) for mat in connectivity_matrix]
    connectivity_matrices_ad.append(connectivity_matrix)

connectivity_matrices_ad = np.stack(connectivity_matrices_ad)

connectivity_matrices_lmci = []

for i in range(42, 78):
    mat = scipy.io.loadmat(f'data_ad_lmci_R/data_ad_lmci_R/LMCI_RH/subject{i}.mat')
    connectivity_matrix = mat['A'][:, :, :]
    connectivity_matrix = [normalize_matrix(mat) for mat in connectivity_matrix]
    connectivity_matrices_lmci.append(connectivity_matrix)

connectivity_matrices_lmci = np.stack(connectivity_matrices_lmci)


In [27]:
connectivity_matrices_ad = []

for i in range(1, 42):
    mat = scipy.io.loadmat(f'data_ad_lmci_L/data_ad_lmci_L/AD_LH/subject{i}.mat')
    connectivity_matrix = mat['A'][:, :, :]
    connectivity_matrix = [normalize_matrix(mat) for mat in connectivity_matrix]
    connectivity_matrices_ad.append(connectivity_matrix)

connectivity_matrices_ad = np.stack(connectivity_matrices_ad)

connectivity_matrices_lmci = []

for i in range(42, 78):
    mat = scipy.io.loadmat(f'data_ad_lmci_L/data_ad_lmci_L/LMCI_LH/subject{i}.mat')
    connectivity_matrix = mat['A'][:, :, :]
    connectivity_matrix = [normalize_matrix(mat) for mat in connectivity_matrix]
    connectivity_matrices_lmci.append(connectivity_matrix)

connectivity_matrices_lmci = np.stack(connectivity_matrices_lmci)


In [28]:
ground_truth_nc_l_list = np.mean(connectivity_matrices_ad, axis=0)
ground_truth_nc_l = np.mean(ground_truth_nc_l_list, axis=2)
np.save('ground_truth_ad_l', ground_truth_nc_l)

In [29]:
ground_truth_nc_l_list = np.mean(connectivity_matrices_lmci, axis=0)
ground_truth_nc_l = np.mean(ground_truth_nc_l_list, axis=2)
np.save('ground_truth_lmci_l', ground_truth_nc_l)